## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-07-23-56-17 +0000,bbc,Savannah Guthrie says 'we will pay' for mother...,https://www.bbc.com/news/articles/ce9y99eg0nno...
1,2026-02-07-23-50-04 +0000,nypost,"SoCal coffee shop hid strip club inside, with ...",https://nypost.com/2026/02/07/us-news/socal-co...
2,2026-02-07-23-48-08 +0000,startribune,Timberwolves respond to Rudy Gobert’s comments...,https://www.startribune.com/timberwolves-respo...
3,2026-02-07-23-46-56 +0000,nyt,Man Pleads Guilty to Arson in 2019 Attack on T...,https://www.nytimes.com/2026/02/07/us/tennesse...
4,2026-02-07-23-42-42 +0000,latimes,"Yes, there really was a 'March for Billionaire...",https://www.latimes.com/california/story/2026-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2533/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
71,trump,27
45,new,12
74,iran,9
85,mayor,9
230,up,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
157,2026-02-07-03-07-00 +0000,wsj,A federal judge has ordered the Trump administ...,https://www.wsj.com/politics/policy/judge-bloc...,73
32,2026-02-07-21-38-40 +0000,nypost,Trump says US held ‘very good talks with Iran’...,https://nypost.com/2026/02/07/world-news/trump...,60
76,2026-02-07-16-52-10 +0000,nypost,Tehran’s foreign minister warns they’re ready ...,https://nypost.com/2026/02/07/world-news/tehra...,60
179,2026-02-07-01-00-00 +0000,wsj,President Trump chose a Federal Reserve chair ...,https://www.wsj.com/economy/central-banking/th...,60
158,2026-02-07-03-03-00 +0000,wsj,The Trump administration announced a new strat...,https://www.wsj.com/politics/national-security...,57


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
157,73,2026-02-07-03-07-00 +0000,wsj,A federal judge has ordered the Trump administ...,https://www.wsj.com/politics/policy/judge-bloc...
5,40,2026-02-07-23-41-35 +0000,nypost,Savannah Guthrie pleads for mom Nancy’s return...,https://nypost.com/2026/02/07/us-news/savannah...
55,34,2026-02-07-19-42-07 +0000,nypost,Rick Caruso blasts Karen Bass — and drops trut...,https://nypost.com/2026/02/07/us-news/rick-car...
43,34,2026-02-07-20-56-48 +0000,nypost,Epstein cultivated close ties with two key Rus...,https://nypost.com/2026/02/07/world-news/epste...
176,34,2026-02-07-01-15-47 +0000,nyt,Super Bowl Visitors Find San Francisco Better ...,https://www.nytimes.com/2026/02/06/us/san-fran...
32,33,2026-02-07-21-38-40 +0000,nypost,Trump says US held ‘very good talks with Iran’...,https://nypost.com/2026/02/07/world-news/trump...
138,31,2026-02-07-09-00-00 +0000,cbc,Should you stay or should you go? Canadians re...,https://www.cbc.ca/news/world/cuba-travel-advi...
179,31,2026-02-07-01-00-00 +0000,wsj,President Trump chose a Federal Reserve chair ...,https://www.wsj.com/economy/central-banking/th...
15,30,2026-02-07-23-09-26 +0000,startribune,The latest: Jessie Diggins finishes eighth in ...,https://www.startribune.com/2026-winter-olympi...
98,30,2026-02-07-13-41-01 +0000,startribune,Immigrant whose skull was broken in eight plac...,https://www.startribune.com/immigrant-whose-sk...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
